# JETANK 游戏手柄控制

如果你有USB接口的游戏手柄的话，可以通过此文档用来实现使用手柄控制JETANK的相关功能。

与Jetbot提供的手柄例程不同，在这里你需要将游戏手柄的USB接收端与Jetson Nano的USB接口相连接，而不是连接到电脑上。

USB接收端与Jetson Nano连接后，打开手柄上面的开关，手柄上面的指示灯亮起，之后再运行以下的代码块。


### 实时画面

运行以下代码打开摄像头画面，当画面打开后，你可以在画面上面点击鼠标右键，选择`Create new view for output`，然后将画面拖拽放到旁边合适的地方即可。

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


# 基础控制相关的代码

由于舵机的连杆逆运动学函数需要被连续调用才可以让机械臂末端以直线运动，所以需要导入threading库来进行机械臂的多线程控制。

In [2]:
import threading
from time import sleep
from jetbot import Robot
from SCSCtrl import TTLServo

robot = Robot()

Succeeded to open the port
Succeeded to change the baudrate



定义一些与基础控制相关的变量和参数。

In [3]:
moveStartTor = 0.15    # 电机起始转动阈值，减速电机在低PWM区间内不能转动，长时间通电堵转会减少电机寿命。
fb_input = 0           # 存储前进方向的速度参数
lr_input = 0           # 存储转向命令的参数

servoCtrlTime = 0.001  # 总线舵机通信后增加一个延迟，避免报错
speedInput = 300       # 摄像头俯仰和水平方向转动的速度

armXStatus = 0         # 机械臂X轴（远近方向）的运动状态
armYStatus = 0         # 机械臂Y轴（垂直方向）的运动状态
cameStatus = 0         # 摄像头运动状态。

goalX = 130            # 存储机械臂运动终点X轴坐标
goalY = 50             # 存储机械臂运动终点Y轴坐标
goalC = 0              # 存储摄像头运动位置。

xMax = 210             # 设置机械臂的X轴最大值
xMin = 140             # 设置机械臂的X轴最小值

yMax = 120             # 设置机械臂的Y轴最大值
yMix = -120            # 设置机械臂的Y轴最小值

cMax = 25              # S设置摄像头的位置最大值。
cMin = -45             # S设置摄像头的位置最小值。
cDan = 0               # S设置摄像头的位置安全临界值。


movingTime = 0.005     # 设置机械臂逆运动学函数每两个相邻位置间的运行时间
movingSpeed = 1        # 设置机械臂的运动速度

grabStatus = 0         # 机械臂夹头舵机的运动状态

cameraPosCheckMode = 1 # 摄像头位置检测功能的开关，1为开启，0为关闭，开启后可避免机械臂与摄像头干涉
cameraPosDanger = 0    # 摄像头是否处在机械臂危险区
cameraMoveSafe = 0     # 摄像头是否已经运动到安全区


运行例程后，机械臂自动运动到（goalX, goalY）位置，其余舵机缓慢转动到初始位置。

In [4]:
TTLServo.xyInputSmooth(goalX, goalY, 3)
TTLServo.servoAngleCtrl(1, 0, 1, 100)
TTLServo.servoAngleCtrl(4, 0, 1, 100)
TTLServo.servoAngleCtrl(5, 0, 1, 100)
sleep(3.5)
print('ready!')

ready!



定义一些基础运动的方法。

In [5]:
# 机械臂停止当前动作
def armStop():
    global armXStatus, armYStatus
    if armXStatus or armYStatus:
        armXStatus = 0
        armYStatus = 0
        TTLServo.servoStop(2)
        sleep(servoCtrlTime)
        TTLServo.servoStop(3)
        sleep(servoCtrlTime)
        print([goalX, goalY])


# 摄像头位置检测和保护功能
def armCameraPosCheck():
    global cameraPosDanger, cameraMoveSafe, goalC
    if goalY < -50 and cameraPosCheckMode:
        cameraPosDanger = 1
        if cameraMoveSafe and goalC < cDan:
            goalC = cDan
            TTLServo.servoAngleCtrl(5, cDan, 1, speedInput)
            sleep(servoCtrlTime)
            cameraMoveSafe = 0
    else:
        cameraPosDanger = 0


# 控制机械臂夹头夹取和松开
def grabCtrlCommand(commandType, valueInput):
    global grabStatus
    if commandType == 'tl':
        commandLoose = 0
        if valueInput:
            commandGrab = 1
        else:
            commandGrab = 0
    elif commandType == 'tl2':
        commandGrab = 0
        if valueInput:
            commandLoose = 1
        else:
            commandLoose = 0
    else:
        return

    if commandGrab and not grabStatus:
        TTLServo.servoAngleCtrl(4, -90, 1, 150)
        grabStatus = 1
        sleep(servoCtrlTime)
    elif commandLoose and not grabStatus:
        TTLServo.servoAngleCtrl(4, 0, 1, 150)
        grabStatus = 1
        sleep(servoCtrlTime)

    if not commandGrab and not commandLoose and grabStatus:
        TTLServo.servoStop(4)
        grabStatus = 0
        sleep(servoCtrlTime)


# 机械臂控制函数，负责命令输入
def armCtrlCommand(commandType, valueInput):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus
    commandXin, commandXde, commandYin, commandYde = 0, 0, 0, 0
    if commandType == 'y':
        if valueInput:
            ctrlArm.moveXin = 1
            ctrlArm.resume()
            print('run')
        else:
            ctrlArm.moveXin = 0
            ctrlArm.pause()
    elif commandType == 'a':
        if valueInput:
            ctrlArm.moveXde = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveXde = 0
            ctrlArm.resume()
    elif commandType == 'tr':
        if valueInput:
            ctrlArm.moveYin = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveYin = 0
            ctrlArm.resume()
    elif commandType == 'tr2':
        if valueInput:
            ctrlArm.moveYde = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveYde = 0
            ctrlArm.resume()
    else:
        return


# 机械臂控制函数，负责动作执行（在另一个线程中）
def armCtrl(commandXin, commandXde, commandYin, commandYde):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus, cameraMoveSafe
    armCameraPosCheck()
    if commandXin:
        armXStatus = 1
        goalX += movingSpeed
        if goalX > xMax:
            goalX = xMax
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    elif commandXde:
        armXStatus = 1
        goalX -= movingSpeed
        if goalX < xMin:
            goalX = xMin
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    if commandYin:
        armYStatus = 1
        goalY += movingSpeed
        if goalY > yMax:
            goalY = yMax
        if goalY > -50:
            cameraMoveSafe = 1
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    elif commandYde:
        armYStatus = 1
        goalY -= movingSpeed
        if goalY < yMix:
            goalY = yMix
        if goalY > -50:
            cameraMoveSafe = 1
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    if not commandXin and not commandXde and not commandYin and not commandYde:
        armStop()


# 摄像头俯仰动作停止
def tiltStop():
    global cameStatus
    cameStatus = 0
    TTLServo.servoStop(5)
    sleep(servoCtrlTime)


# 摄像头（机械臂）水平方向动作停止
def panStop():
    TTLServo.servoStop(1)
    sleep(servoCtrlTime)

    
# 单独线程中的摄像头俯仰控制。
def cameraTiltCtrl(commandCin, commandCde):
    global goalC, cameStatus
    if commandCde:
        cameStatus = 1
        if not cameraPosDanger:
            goalC -= movingSpeed
            if goalC < cMin:
                goalC = cMin
            TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
            sleep(servoCtrlTime)
        else:
            goalC -= movingSpeed
            if goalC < cDan:
                goalC = cDan
            TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
            sleep(servoCtrlTime)
        return
    if commandCin:
        cameStatus = 1
        goalC += movingSpeed
        if goalC > cMax:
            goalC = cMax
        TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
        sleep(servoCtrlTime)
        return
    if not commandCde and not commandCin:
        tiltStop()


# 摄像头（机械臂）水平方向转动控制
def cameraPanCtrlCommand(commandType, valueInput, buttonLoop):
    commandLeft, commandRight = 0, 0
    if commandType == 'lr' and buttonLoop:
        if valueInput == -1:
            commandLeft = 1
        elif valueInput == 1:
            commandRight = 1
    elif commandType == 'x' and not buttonLoop:
        if valueInput:
            commandLeft = 1
    elif commandType == 'b' and not buttonLoop:
        if valueInput:
            commandRight = 1
    else:
        return

    if commandLeft:
        TTLServo.servoAngleCtrl(1, -80, 1, speedInput)
        sleep(servoCtrlTime)
    elif commandRight:
        TTLServo.servoAngleCtrl(1, 80, 1, speedInput)
        sleep(servoCtrlTime)
    elif not commandLeft and not commandRight:
        panStop()


# 摄像头俯仰动作控制
def cameraTiltCtrlCommand(commandType, valueInput):
    commandUp, commandDown = 0, 0
    if commandType == 'fb':
        if valueInput == -1:
            commandUp = 1
        elif valueInput == 1:
            commandDown = 1
    else:
        return

    if commandUp:
        ctrlArm.moveCde = 1
        ctrlArm.moveCin = 0
        ctrlArm.resume()
    elif commandDown:
        ctrlArm.moveCin = 1
        ctrlArm.moveCde = 0
        ctrlArm.resume()
    else:
        ctrlArm.moveCin = 0
        ctrlArm.moveCde = 0
        ctrlArm.resume()


# 电机停止转动
def moveStop():
    global fb_input, lr_input
    fb_input = 0
    lr_input = 0
    robot.stop()


# 机器人移动控制
def moveSmoothCtrl(commandType, valueInput):
    global fb_input, lr_input
    if commandType == 'y':
        fb_input = valueInput
    elif commandType == 'x':
        lr_input = valueInput
    else:
        return

    if abs(fb_input) < moveStartTor and abs(lr_input) < moveStartTor:
        moveStop()
        return
    leftSpeed  = -fb_input + lr_input
    rightSpeed = -fb_input - lr_input
    robot.left_motor.value  = leftSpeed
    robot.right_motor.value = rightSpeed


机械臂的多线程控制对象

In [6]:
# 机械臂的多线程控制对象，实例化后使用pause()和resume()来控制run()的暂停与恢复
class ArmCtrlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(ArmCtrlThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()
        self.moveXin = 0
        self.moveXde = 0
        self.moveYin = 0
        self.moveYde = 0
        self.moveCin = 0
        self.moveCde = 0

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()
            armCtrl(self.moveXin, self.moveXde, self.moveYin, self.moveYde)
            if self.moveCin or self.moveCde:
                cameraTiltCtrl(self.moveCin, self.moveCde)
            if not self.moveXin and not self.moveXde and not self.moveYin and not self.moveYde and not self.moveCin and not self.moveCde:
                self.pause()
            sleep(movingTime)


# 实例化并开启多线程的舵机控制对象
ctrlArm = ArmCtrlThread()
ctrlArm.start()


# 手柄控制相关的代码

In [7]:
import os, struct, array
from fcntl import ioctl
 
# Iterate over the joystick devices.
#手柄接收器连接Jetson Nano后，/dev/input/内的设备名称一般为js0，这里查找所有名字内右js的设备
print('Available devices:')
for fn in os.listdir('/dev/input'):
    if fn.startswith('js'):
        print('  /dev/input/%s' % (fn))
 
 
# These constants were borrowed from linux/input.h
# 以下内容来自linux/input.h
axis_names = {
    0x00 : 'x',
    0x01 : 'y',
    0x02 : 'z',
    0x03 : 'rx',
    0x04 : 'ry',
    0x05 : 'rz',
    0x06 : 'trottle',
    0x07 : 'rudder',
    0x08 : 'wheel',
    0x09 : 'gas',
    0x0a : 'brake',
    0x10 : 'lr',
    0x11 : 'fb',
    0x12 : 'hat1x',
    0x13 : 'hat1y',
    0x14 : 'hat2x',
    0x15 : 'hat2y',
    0x16 : 'hat3x',
    0x17 : 'hat3y',
    0x18 : 'pressure',
    0x19 : 'distance',
    0x1a : 'tilt_x',
    0x1b : 'tilt_y',
    0x1c : 'tool_width',
    0x20 : 'volume',
    0x28 : 'misc',
}
 
button_names = {
    0x120 : 'trigger',
    0x121 : 'thumb',
    0x122 : 'thumb2',
    0x123 : 'top',
    0x124 : 'top2',
    0x125 : 'pinkie',
    0x126 : 'base',
    0x127 : 'base2',
    0x128 : 'base3',
    0x129 : 'base4',
    0x12a : 'base5',
    0x12b : 'base6',
    0x12f : 'dead',
    0x130 : 'a',
    0x131 : 'b',
    0x132 : 'c',
    0x133 : 'x',
    0x134 : 'y',
    0x135 : 'z',
    0x136 : 'tl',
    0x137 : 'tr',
    0x138 : 'tl2',
    0x139 : 'tr2',
    0x13a : 'select',
    0x13b : 'start',
    0x13c : 'mode',
    0x13d : 'thumbl',
    0x13e : 'thumbr',
 
    0x220 : 'dpad_up',
    0x221 : 'dpad_down',
    0x222 : 'dpad_left',
    0x223 : 'dpad_right',
 
    0x2c0 : 'dpad_left',
    0x2c1 : 'dpad_right',
    0x2c2 : 'dpad_up',
    0x2c3 : 'dpad_down',
}
 
axis_map = []
button_map = []
 
# Open the joystick device.
# 打开游戏手柄设备
fn = '/dev/input/js0'
print('Opening %s...' % fn)
jsdev = open(fn, 'rb')
 
# Get the device name.
# 获得设备名称
buf = array.array('B', [0] * 64)
ioctl(jsdev, 0x80006a13 + (0x10000 * len(buf)), buf) # JSIOCGNAME(len)
js_name = buf.tobytes().rstrip(b'\x00').decode('utf-8')
print('Device name: %s' % js_name)
 
# Get number of axes and buttons.
# 获得按键和遥感的数量。
buf = array.array('B', [0])
ioctl(jsdev, 0x80016a11, buf)
num_axes = buf[0]
 
buf = array.array('B', [0])
ioctl(jsdev, 0x80016a12, buf)
num_buttons = buf[0]
 
# Get the axis map.
# 获得遥感地址并与axis_names中的名称对应
buf = array.array('B', [0] * 0x40)
ioctl(jsdev, 0x80406a32, buf) # JSIOCGAXMAP
 
for axis in buf[:num_axes]:
    axis_name = axis_names.get(axis, 'unknown(0x%02x)' % axis)
    axis_map.append(axis_name)
 
# Get the button map.
# 获得按键地址并与axis_names中的名称对应
buf = array.array('H', [0] * 200)
ioctl(jsdev, 0x80406a34, buf) # JSIOCGBTNMAP
 
for btn in buf[:num_buttons]:
    btn_name = button_names.get(btn, 'unknown(0x%03x)' % btn)
    button_map.append(btn_name)
 
print('%d axes found: %s' % (num_axes, ', '.join(axis_map)))
print('%d buttons found: %s' % (num_buttons, ', '.join(button_map)))
 

# 读取手柄变化信息并控制机器人的线程  
class inputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(inputThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()
            evbuf = jsdev.read(8)
            if evbuf:
                time, value, type, number = struct.unpack('IhBB', evbuf)

                if type & 0x80:
                     print("(initial)", end="")

                if type & 0x01:
                    button = button_map[number]
                    if button:
                        grabCtrlCommand(button, value)
                        armCtrlCommand(button, value)
                        cameraPanCtrlCommand(button, value, 0)

                if type & 0x02:
                    axis = axis_map[number]
                    if axis:
                        moveSmoothCtrl(axis, value/32767.0)
                        cameraPanCtrlCommand(axis, int(round(value/32767.0,0)), 1)
                        cameraTiltCtrlCommand(axis, int(round(value/32767.0,0)))


# 实例化手柄输入线程并开始读取信息
inputThreading = inputThread()
inputThreading.start()
inputThreading.resume()

Available devices:
  /dev/input/js0
Opening /dev/input/js0...
Device name: ShanWan PC/PS3/Android
8 axes found: x, y, z, rz, gas, brake, lr, fb
16 buttons found: a, b, c, x, y, z, tl, tr, tl2, tr2, select, start, mode, thumbl, thumbr, unknown(0x13f)
(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)run
[140, 50]
run
run
[140, 77]
[140, 11]
[140, 12]


# 按键说明

L1： 夹头夹紧。  
L2： 夹头松开。
A/B/X/Y键： 机械臂末端点在前后方向上向后收回/摄像头向右转动/摄像头向左转动/机械臂末端点在前后方向向前伸出。  
左操作遥感： 控制机器人的前后运动和转向。  
左侧方向按键（上）： 摄像头向上转动。  
左侧方向按键（下）： 摄像头向下转动。  
左侧方向按键（左）： 摄像头向左转动。  
左侧方向按键（右）： 摄像头向右转动。  
R1键： 机械臂末端点在竖直方向上向上运动。  
R2键： 机械臂末端点在竖直方向上向下运动。  

在结束该例程前，需要关闭摄像头，这样才可以释放摄像头资源供其它例程使用。

In [8]:
camera.stop()